In [63]:
import pandas as pd
import re

In [64]:
datadir = '/home/xavier/data/'
clinvar_file = 'variant_summary_02.txt' # Updated 5/21/2018

# First thing, download and survey the 'Clinvar' data

## ➜ ClinVar is a freely accessible, public archive of reports of the relationships among human variations and phenotypes, with supporting evidence.

+ A phenotype results from the expression of an organism's genetic code, its genotype, as well as the influence of environmental factors and the interactions between the two. [Wikipedia](https://en.wikipedia.org/wiki/Phenotype)
+ File is from **[THIS FTP DIRECTORY](ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/)** (which is updated monthly)
+ ** Download this file [Variant_summary.txt](ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz)** which is a 809K row dataset listing 

In [65]:
clinvar = pd.read_csv(datadir + clinvar_file, sep='\t', lineterminator='\n')

/home/xavier/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
# count the rows of data
clinvar.shape

(811417, 31)

## ...⇧
> **811K Rows of data:** The Variant Summary updated on 5/21 is 2K rows longer than the previous

+ Previous was 809K rows in this dataset.

- - -

In [67]:
# What are the Reference Genomes in this data?
clinvar.groupby('Assembly').size()

Assembly
GRCh37    397447
GRCh38    389089
NCBI36     19545
dtype: int64

In [68]:
397447 + 389089 + 19545

806081

## ...⇧
> **806K Rows of "Reference Genome" data:** This is an anomaly!!

+ Data has **811K** rows
+ Yet the Assembly counts **only 806K** rows

In [69]:
clinvar.Assembly

0         GRCh37
1         GRCh38
2         GRCh37
3         GRCh38
4         GRCh37
5         GRCh38
6         GRCh37
7         GRCh38
8         GRCh37
9         GRCh38
10        GRCh37
11        GRCh38
12        NCBI36
13        GRCh37
14        GRCh38
15        GRCh38
16        GRCh37
17        GRCh37
18        GRCh38
19        GRCh37
20        GRCh38
21        GRCh37
22        GRCh38
23        GRCh37
24        GRCh38
25        GRCh37
26        GRCh38
27        GRCh37
28        GRCh38
29        GRCh37
           ...  
811387    GRCh38
811388    GRCh37
811389    GRCh38
811390    GRCh38
811391    GRCh37
811392    GRCh37
811393    GRCh38
811394    GRCh37
811395    GRCh38
811396    GRCh38
811397    GRCh37
811398    GRCh38
811399    GRCh37
811400    GRCh38
811401    GRCh37
811402    GRCh37
811403    GRCh38
811404    GRCh38
811405    GRCh37
811406    GRCh38
811407    GRCh37
811408    GRCh38
811409    GRCh37
811410       NaN
811411       NaN
811412       NaN
811413       NaN
811414       N

## ...⇧
> **~5K Rows of "Reference Genome" are neither 36, 37, or 38:**

+ For example, the NaN values in the last few rows listed above.
+ So will only work with those explicitly referenced as Human Reference Genome #38

In [70]:
# Work with Human Reference Genome #38
latest = clinvar[clinvar['Assembly'] == 'GRCh38']
latest.shape

(389089, 31)

## ...⇧

> **389K Unfiltered rows of data fir Reference Genome #38**

- - -

## Now let's cross-reference some count of this data using other tools

+ The [ClinVar Advanced Search Builder](https://www.ncbi.nlm.nih.gov/clinvar/advanced/) allows you to slice Clinvar data using custom variables
+ We're interested in the most authoritative data.

> Let's start with "multiple submitters, no conflict." Should be **~53K rows**

![Should be ~ 53K rows](https://i.imgur.com/2cmVd4jl.png)


- - -

In [71]:
# Let's double check some stats on this dataframe:
latest.groupby('ReviewStatus').size()


ReviewStatus
criteria provided, conflicting interpretations           18415
criteria provided, multiple submitters, no conflicts     52969
criteria provided, single submitter                     249069
no assertion criteria provided                           48076
no assertion provided                                    10707
no interpretation for the single variant                   656
practice guideline                                          23
reviewed by expert panel                                  9174
dtype: int64

## ...⇧

> **Matching up pretty well**

+ Discrepancies of 'No Conflicts and 'Reviewed by Experts' can be explained by the NaN values are filtered out of our dataset, but remain in the 'Advanced Search Builder'

![Matchup](https://i.imgur.com/pswXi9Zl.jpg)

In [72]:
# Further filter the dataset down by these 'authoritative' flags
# from the 'Review Status' column

authoritative = latest[
    (latest['ReviewStatus'] == 'criteria provided, multiple submitters, no conflicts') | 
    (latest['ReviewStatus'] == 'practice guideline') | 
    (latest['ReviewStatus'] == 'reviewed by expert panel') 
]
authoritative.shape

(62166, 31)

In [73]:
# double check
52969 + 23 + 9174

62166

In [74]:
# percentage of total Ref #38 data
(62166 / 389089) * 100

15.97732138405352

## ...⇧

> **62K Rows of 'Authoritative' reviews**

+ So that means that of the original **389K** rows of data, on **62K** rows of data are 'meaningful...
+ Only **16%** of this data is 'authoritative', based on our filters.

- - -

# Idea for marketing

> Show the rise in 'authoritative' data on clinical signifigance by applying this exercise back in time on retrospective datasets. ➜ This would demonstrate how rapidly authoritative correlations in markers are rising, over a period of months. (Nice little marketing tidbit.)

![marketing ideas](https://i.imgur.com/gshsM2Zm.png?1)

## Time to start matching the OMIM tags

+ First order is to find OMIM tags that match the **59 ACMG Recommendations Diseases**
+ Import this file: [ACMG Recommendations for Reporting of Incidental Findings in Clinical Exome and Genome Sequencing](https://www.ncbi.nlm.nih.gov/clinvar/docs/acmg/)

In [75]:
# Import Disease names and OMIM tags and convert to a list
acmg = pd.read_csv(datadir + 'ACMG_Conditions_OMIM.csv')
acmgtags = acmg.ACMGTAGS.tolist()

In [76]:
# function to write OMIM TAGS to a new Column
def omim_tags (x):
    # Find anything that has an OMIM tag, from 1 to 6 numerals long
    res = re.findall(r"(?:OMIM:(\d{1,6}))",x)
    if res:
        return res
    else:
#         print("NA")
         return ("NA")

In [77]:
# function to write "OMIM > ACMG" matches to a new Column
def acmg_matches (x):
    # Find anything that has an OMIM tag, from 4 to 6 numerals long
    res = re.finditer(r"(?:OMIM:(\d{4,6}))",x)
    if res:
        for hit in res:
#             print (int(hit.group(1)))
            # Check if matches against ACMG
            if int(hit.group(1)) in acmgtags:
                return int(hit.group(1))
        else:
            return("NA")
#          return ("NA")

In [78]:
authoritative['OMIMTAGS'] = authoritative.PhenotypeIDS.apply(omim_tags)

/home/xavier/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [79]:
authoritative['ACMGTAGS'] = authoritative.PhenotypeIDS.apply(acmg_matches)

/home/xavier/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
onlyomim = authoritative[authoritative['OMIMTAGS'] != "NA"]
onlyacmg = authoritative[authoritative['ACMGTAGS'] != "NA"]

In [81]:
# write 1500 rows to a swapfile
onlyacmg.iloc[0:1499].to_csv(datadir + 'acmg_matches-1500.csv')

In [82]:
onlyacmg.shape

(12138, 33)

## ...⇧
> **12K rows Match to ACMG Diseases**:

# 20% of the authoritative data matches an ACMG condition

In [83]:
# percentage data that matches ACMG
(onlyacmg.shape[0]/authoritative.shape[0])*100

19.525142360775988

# There are definitely instances where multiple ACMG diseases match against a single row of our Ref #38, authoritative data
## But let's finish strongand ONLY LIST THE FIRST CONDITION
## Merging the ACMG disease name

In [85]:
result = pd.merge(onlyacmg,acmg,on='ACMGTAGS',how='left')

In [86]:
result.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,OMIMTAGS,ACMGTAGS,DISEASE
0,15440,single nucleotide variant,NM_017841.2(SDHAF2):c.232G>A (p.Gly78Arg),54949,SDHAF2,HGNC:26034,Pathogenic,1,"Sep 06, 2017",113560320,...,"criteria provided, multiple submitters, no con...",4,"ACMG2013,ACMG2016",N,"OMIM Allelic Variant:613019.0001,UniProtKB (pr...",3,401,[601650],601650,Paragangliomas 2
1,15773,single nucleotide variant,NM_024334.2(TMEM43):c.1073C>T (p.Ser358Leu),79188,TMEM43,HGNC:28472,Pathogenic,1,"Jun 09, 2017",63750743,...,"criteria provided, multiple submitters, no con...",8,"ACMG2013,ACMG2016",N,"OMIM Allelic Variant:612048.0001,UniProtKB (pr...",3,734,[604400],604400,Arrhythmogenic right ventricular cardiomyopath...
2,15837,single nucleotide variant,NM_000038.5(APC):c.904C>T (p.Arg302Ter),324,APC,HGNC:583,Pathogenic,1,"Apr 22, 2017",137854568,...,"criteria provided, multiple submitters, no con...",8,"ACMG2013,ACMG2016",N,"HGMD:CM910029,OMIM Allelic Variant:611731.0002...",3,798,[175100],175100,Adenomatous polyposis coli
3,15840,single nucleotide variant,NM_000038.5(APC):c.4012C>T (p.Gln1338Ter),324,APC,HGNC:583,Pathogenic/Likely pathogenic,1,"Mar 22, 2016",121913327,...,"criteria provided, multiple submitters, no con...",4,"ACMG2013,ACMG2016",N,OMIM Allelic Variant:611731.0009,3,801,"[114500, 175100]",175100,Adenomatous polyposis coli
4,15845,single nucleotide variant,NM_000038.5(APC):c.1621C>T (p.Gln541Ter),324,APC,HGNC:583,Pathogenic,1,"May 24, 2017",137854572,...,"criteria provided, multiple submitters, no con...",4,"ACMG2013,ACMG2016",N,OMIM Allelic Variant:611731.0014,3,806,[175100],175100,Adenomatous polyposis coli


In [87]:
result.to_csv(datadir + 'acmg_authoritative_matches_full.csv')

In [88]:
onlyomim.shape

(38590, 33)

## ...⇧
> **38K rows contain OMIM tags**: 62% of total "Expert Reviewed, No Conflict" data

+ ...that's a lot of data loss, because it doesn't share the OMIM tag!
+ We may want to come back and try to match by disease ddescription instead of just the OMIM TAG

In [84]:
# percentage data sliced away by not using OMIM tags
(onlyomim.shape[0]/authoritative.shape[0])*100

62.075732715632334

In [89]:
# create a column that counts the number of matches
onlyomim["MATCHCOUNT"]=onlyomim["OMIMTAGS"].apply(lambda x: len(x))

/home/xavier/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [90]:
# Table of the number of matches
onlyomim.groupby('MATCHCOUNT').size()

MATCHCOUNT
1     31562
2      4628
3      1164
4       383
5       688
6        57
7        48
8        21
9         9
10       21
11        4
12        1
13        2
14        1
19        1
dtype: int64

## ...⇧
> **Signifigant number have multiple OMIM tags**: 

+ 12% of rows have 2 OMIM tags
+ 3% of rows have 3 OMIM tags


In [91]:
(1164/38590)*100

3.0163254729204456

In [92]:
(4628/38590)*100

11.992744234257579

## NOTES FROM ALICE

+ Expert Panel, Practice Guideline, multiple submitters, no conflict
+ Check out information on drug response
    + Add  Drug Response (clniincial signifigance > Drug Response)
+ Assembly
    + Only Keep 38


